In [ ]:
# Instalação e configuração de variaveis ambiente para utilizar Google Drive
# Se utilizar o Colab ajustar para True para instalação dos pre-requisitos
import os
import sys

colab = True

if colab==True:
  from google.colab import drive
  drive.mount('/content/drive')

  # Instalação de requisitos
  !apt-get update # Update apt-get repository.
  !apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
  !wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
  !tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
  !pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

  # Definindo a variável de ambiente do Java
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  # Definindo a variável de ambiente do Spark
  #os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"
  os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

  # Variaveis de configuração
  # Diretorio base dos dados
  # Google Drive
  dir_base = "/content/drive/MyDrive/jupyter/pcd_0124_analise_de_credito/data/"
else:
  # Variaveis de configuração
  # Diretorio base dos dados
  # Local PC
  dir_base = "data/"

# Importando a findspark
import findspark

# Iniciando o findspark
findspark.init()
from  pyspark.sql import  SparkSession
spark = SparkSession \
    .builder \
    .appName("Análise de Crédito - Previous Application") \
    .getOrCreate()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

Mounted at /content/drive
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,920 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,358 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,104 kB]
Hit:13 https://ppa.laun

In [ ]:
#Importação das bibliotecas para utilização no processo de Feature Selection

import pandas as pd
import numpy as np
import random
import gc
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier

# Definindo a semente
random.seed(123)

In [ ]:
# Funções auxiliares para agilizar o tratamento dos dados:
# - Criação de metadados referente ao conjunto de dados
# - Remoção de features altamente correlacionadas
# - Amostragem
# - Variancia
# - Feature Importance

def pod_academy_generate_metadata(dataframe):
    """
    Gera um dataframe contendo metadados das colunas do dataframe fornecido.

    :param dataframe: DataFrame para o qual os metadados serão gerados.
    :return: DataFrame contendo metadados.
    """

    # Coleta de metadados básicos
    metadata = pd.DataFrame({
        'nome_variavel': dataframe.columns,
        'tipo': dataframe.dtypes,
        'qt_nulos': dataframe.isnull().sum(),
        'percent_nulos': round((dataframe.isnull().sum() / len(dataframe))* 100,2),
        'cardinalidade': dataframe.nunique(),
    })
    metadata=metadata.sort_values(by='percent_nulos',ascending=False)
    metadata = metadata.reset_index(drop=True)

    return metadata

def remove_highly_correlated_features(df, threshold):
  # Calculate the correlation matrix
  corr_matrix = df.corr().abs()

  # Select the upper triangle of the correlation matrix
  upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

  # Identify columns to drop based on the threshold
  to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)]

  # Drop the columns
  df_reduced = df.drop(columns=to_drop)

  return df_reduced, to_drop

def amostragem(df,tamanho_amostra):
  amostra = df.sample(n=tamanho_amostra,random_state=42)
  return amostra

def variancia(table_01, threshold):
    cat_attributes = table_01.select_dtypes(include='object')
    num_attributes = table_01.select_dtypes(exclude='object')

    # Preparação dos dados

    # Create a label encoder
    label_encoder = LabelEncoder()

    # Fit and transform the data
    for obj in cat_attributes.columns:
        cat_attributes[obj] = label_encoder.fit_transform(cat_attributes[obj].astype(str))

    table_final = pd.concat([num_attributes,cat_attributes],axis=1)

    # Scaler
    scaler = StandardScaler()

    table_final_02 = scaler.fit_transform(table_final)

    table_final_03 = pd.DataFrame(table_final_02,columns=table_01.columns)

    selector = VarianceThreshold(threshold)
    selector.fit_transform(table_final_03)

    # Colunas selecionadas
    selected_features = table_final_03.columns[selector.get_support()]

    # Manter apenas features selecionadas
    table_final_03_select01 = table_final_03[selected_features]

    return table_final_03_select01

def feature_importance(df):
  # Amostragem
  amostra = df.sample(n=85000,random_state=42)

  X = amostra.drop(columns=['SK_ID_CURR','TARGET'])
  y = amostra.TARGET
  id = amostra.SK_ID_CURR

  # Tratamento de missings
  from sklearn.impute import SimpleImputer
  imputer_num = SimpleImputer(strategy='mean')
  imputer_cat = SimpleImputer(strategy='most_frequent')
  cat_attributes = X.select_dtypes(include='object')
  num_attributes = X.select_dtypes(exclude='object')
  num_imputed = imputer_num.fit_transform(num_attributes)
  cat_imputed = imputer_cat.fit_transform(cat_attributes)

  df_num = pd.DataFrame(num_imputed,columns=num_attributes.columns)
  df_cat = pd.DataFrame(cat_imputed,columns=cat_attributes.columns)

  # Aplicação de encoding
  label_encoder = LabelEncoder()

  # Fit and transform the data
  for obj in cat_attributes.columns:
      df_cat[obj] = label_encoder.fit_transform(df_cat[obj].astype(str))

  y.index = df_num.index
  id.index = df_num.index
  df_tratado = pd.concat([df_num,df_cat,y],axis=1)

  X = df_tratado.drop(columns='TARGET')
  y = df_tratado.TARGET

  # Treino utilizando o Gradient Boosting
  algoritmo = GradientBoostingClassifier(random_state=0)
  algoritmo.fit(X,y)
  feature_importances = algoritmo.feature_importances_

  # Importância das variáveis de acordo com o algoritmo

  df_importancias = pd.DataFrame(X.columns,columns=['Variável'])
  df_importancias['Importância'] = feature_importances

  df_importancias = df_importancias[df_importancias['Importância'] > 0]
  return df_importancias

def vars_selection(df,percentual_preenchimento,threshold,tamanho_amostragem):
  variaveis_publico = df.filter(like='_publico').columns.to_list()
  df_aux = df.loc[:, ~df.columns.str.endswith('_publico')]
  amostra = amostragem(df_aux, tamanho_amostragem)

  metadata_df = pod_academy_generate_metadata(amostra)

  # Avaliar preenchimento
  vars = metadata_df[metadata_df.percent_nulos <= percentual_preenchimento]['nome_variavel']
  # Avaliar correlaçao
  df_reduced, dropped_features = remove_highly_correlated_features(amostra[vars], threshold=threshold)
  # Avaliar variância - remover variáveis constantes
  table_01 = variancia(df_reduced, threshold=0)

  vars_selected = table_01.columns.to_list()
  vars_df_final = vars_selected + variaveis_publico
  df_selected = df[vars_df_final]

  df_importancias = feature_importance(df_selected)

  return df_importancias

In [ ]:
# Leitura dos dados de treino para utilização pela etapa de Feature Selection
# Nesse ponto os dados já foram preparados paa utilizarmos técnicas de seleção de variáveis
# Dentre as técnicas anterioes foram criadas varias colunas de agregação para gerar novos dados.

base_final = "application_train_full_3.parquet"
df_treino_full = pd.read_parquet(dir_base + base_final, engine='auto')


In [ ]:
df_treino_full.shape

(215257, 7852)

In [ ]:
# Seleção de variaveis
# Foi fornecido a base de treino, definido o percentual minimo que as colunas precisam ter de preenchimento,
# caso contrário podiam ser eliminadas, fornecido o valor para que o algoritmo possa definir o maximo aceitável de correlação,
# e também o tamanho da amostragem desejada
# Para seleção de variaveis foi escolhido o GradientBoostingClassifier com feature importance

df = vars_selection(df_treino_full,percentual_preenchimento=80,threshold=0.5,tamanho_amostragem=85000)

<ipython-input-3-ec58bc04a946>:31: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = df.corr().abs()


In [ ]:
# Manter apenas as variáveis selecionadas pelo algoritmo e paramétros definidos
# Na seleção de variáveis foi adicionado a coluna ID e o TARGET para uso posterior no modelo.
# Gerado o arquivo com as variaveis selecionadas na base de treino

vars_selected = df['Variável'].to_list()
vars_selected.append('SK_ID_CURR')
vars_selected.append('TARGET')
df_treino_full[vars_selected].to_csv(dir_base + 'bases_tratadas/treino_full_vars_selecionadas.csv', index=False)

In [ ]:
# Checagem de como ficou o arquivo
df.shape

(134, 2)

In [ ]:
# Leitura dos Dados de Teste para execução dos mesmos passos antiores
base_final = "application_test_full_3.parquet"
df_teste_full = pd.read_parquet(dir_base + base_final, engine='auto')

In [ ]:
# checagem de como está atualmente a base de teste
df_teste_full.shape

(92254, 7851)

In [ ]:
# Realizado o mesmo procedimento na base de teste para ter as mesmas colunas/variaveis selecionadas com exceção do TARGET
vars_selected.remove('TARGET')
df_teste_full[df['Variável']].to_csv(dir_base + 'bases_tratadas/teste_full_vars_selecionadas.csv', index=False)

In [ ]:
# Apenas uma visão geral de como ficou a lista de variaveis selecionadas até o momento
# para serem utilizadas pelo modelo juntamente com ID e TARGET
vars_selected = df['Variável'].to_list()
vars_selected.append('SK_ID_CURR')
vars_selected.append('TARGET')
vars_selected

['VL_MED_AMT_GOODS_PRICE_ULTIMOS_6_MESES_PREVIOUS_APPLICATION',
 'VL_MIN_HOUR_APPR_PROCESS_START_ULTIMOS_6_MESES_PREVIOUS_APPLICATION',
 'VL_MIN_QT_MIN_AMT_PAYMENT_ULTIMOS_3_MESES_INSTALLMENTS_ULTIMOS_12_MESES_PREVIOUS_APPLICATION',
 'QT_MAX_QT_MIN_DAYS_ENTRY_PAYMENT_ULTIMOS_3_MESES_INSTALLMENTS_ULTIMOS_12_MESES_PREVIOUS_APPLICATION',
 'VL_MAX_QT_TOT_NUM_INSTALMENT_NUMBER_ULTIMOS_3_MESES_INSTALLMENTS_ULTIMOS_12_MESES_PREVIOUS_APPLICATION',
 'VL_MAX_QT_MIN_NUM_INSTALMENT_VERSION_ULTIMOS_3_MESES_INSTALLMENTS_ULTIMOS_12_MESES_PREVIOUS_APPLICATION',
 'VL_TOT_NFLAG_INSURED_ON_APPROVAL_ULTIMOS_12_MESES_PREVIOUS_APPLICATION',
 'VL_TOT_QT_MIN_AMT_CREDIT_SUM_ULTIMOS_6_MESES',
 'VL_MIN_QT_MAX_AMT_CREDIT_SUM_ULTIMOS_6_MESES',
 'VL_MED_QT_MIN_AMT_CREDIT_SUM_ULTIMOS_6_MESES',
 'QT_MAX_QT_MIN_DAYS_CREDIT_UPDATE_ULTIMOS_6_MESES',
 'QT_MAX_QT_MAX_DAYS_CREDIT_UPDATE_ULTIMOS_6_MESES',
 'VL_MAX_QT_MED_AMT_CREDIT_SUM_OVERDUE_ULTIMOS_6_MESES',
 'QT_MIN_QT_MIN_DAYS_CREDIT_UPDATE_ULTIMOS_6_MESES',
 'VL_MIN_Q

In [ ]:
print('Fim!')

Fim!
